# Database

This notebook shows how to use a database for storing and loading simulation results.
The interface employs SQLAlchemy, which is installed if you supplied the `[database]` option during gdsfactory installation.

## Overview
1. We create an ad-hoc SQLite database, which will store data in a single file (`database.db` in this case)
2. We show how to add simulation data to the database
3. We use a TODO SQL or redis Docker container for hosting the database oursel/ves. This method may be easily employed for multiple users
4. More scalable database is employed using Litestream. This _streams_ the SQLite database to Amazon, Azure, Google Cloud or a similar online database.


TODO
simple simulation 'cached' to database

first with sqlite?

then with adhoc with docker

then to mongodb or such

(bonus) json???

In [1]:
import gdsfactory as gf
import gdsfactory.database.models as gm

from sqlalchemy import create_engine, text
from sqlalchemy.orm import Session

2023-01-29 19:33:36.096 | INFO     | gdsfactory.config:<module>:50 - Load 'C:\\Users\\Niko\\Dev\\gdsfactory\\gdsfactory' 6.25.2
2023-01-29 19:33:36.625 | INFO     | gdsfactory.technology.layer_views:__init__:779 - Importing LayerViews from KLayout layer properties file: C:\Users\Niko\Dev\gdsfactory\gdsfactory\generic_tech\klayout\tech\layers.lyp.


`gm.metadata` houses the gdsfactory-specific models

In [2]:
engine = create_engine("sqlite:///database.db", echo=True, future=True)
gm.metadata.create_all(engine)

2023-01-29 19:33:36,848 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-29 19:33:36,848 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("process")
2023-01-29 19:33:36,848 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-29 19:33:36,849 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("unit")
2023-01-29 19:33:36,850 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-29 19:33:36,851 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("wafer")
2023-01-29 19:33:36,851 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-29 19:33:36,852 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("computed_result")
2023-01-29 19:33:36,852 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-29 19:33:36,853 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("result")
2023-01-29 19:33:36,854 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-01-29 19:33:36,854 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("reticle")
2023-01-29 19:33:36,854 INFO sqlalchemy.engine.Engine [raw sql] 

In [3]:
c = gf.c.ring_single(radius=10)

In [4]:
with Session(engine) as session:

    w1 = gm.Wafer(name="12", serial_number="ABC")
    r1 = gm.Reticle(name="sky1", wafer_id=w1.id, wafer=w1)
    d1 = gm.Die(name="d00", reticle_id=r1.id, reticle=r1)
    c1 = gm.Component(name=c.name, die_id=d1.id, die=d1)

    print(d1.reticle.wafer)

    component_settings = []

    for key, value in c.settings.changed.items():
        s = gm.ComponentInfo(component=c1, component_id=c1.id, name=key, value=value)
        component_settings.append(s)

    for port in c.ports.values():
        s = gm.Port(
            component=c1,
            component_id=c1.id,
            port_type=port.port_type,
            name=port.name,
            orientation=port.orientation,
            position=port.center,
        )
        component_settings.append(s)

    # add objects
    session.add_all([w1, r1, d1, c1])
    session.add_all(component_settings)

    # flush changes to the database
    session.commit()


2023-01-29 19:33:36,981 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-29 19:33:36,983 INFO sqlalchemy.engine.Engine INSERT INTO wafer (serial_number, name, description) VALUES (?, ?, ?)
2023-01-29 19:33:36,984 INFO sqlalchemy.engine.Engine [generated in 0.00095s] ('ABC', '12', None)
2023-01-29 19:33:36,986 INFO sqlalchemy.engine.Engine INSERT INTO reticle (name, position, size, wafer_id, description) VALUES (?, ?, ?, ?, ?)
2023-01-29 19:33:36,987 INFO sqlalchemy.engine.Engine [generated in 0.00049s] ('sky1', None, None, 21, None)
2023-01-29 19:33:36,988 INFO sqlalchemy.engine.Engine INSERT INTO die (reticle_id, name, position, size, description) VALUES (?, ?, ?, ?, ?)
2023-01-29 19:33:36,988 INFO sqlalchemy.engine.Engine [generated in 0.00036s] (21, 'd00', None, None, None)
2023-01-29 19:33:36,990 INFO sqlalchemy.engine.Engine INSERT INTO component (die_id, name, description) VALUES (?, ?, ?)
2023-01-29 19:33:36,990 INFO sqlalchemy.engine.Engine [generated in 0.00057s] (21, 'r

## Querying the database

In this section, we show different ways to query the database using SQLAlchemy.

Individual rows of a selected model, in this case `Wafer`, from the database are fetched as follows:

In [5]:
with Session(engine) as session:

    # Two ways to do the same thing
    for wafer in session.query(gm.Wafer):
        print(wafer.name, wafer.serial_number)

    for wafer_name, wafer_serial in session.query(gm.Wafer.name, gm.Wafer.serial_number):
        print(wafer_name, wafer_serial)

    # Get the `Wafer` from a child `Reticle`
    for reticle in session.query(gm.Reticle).all():
        print(reticle.name, reticle.wafer.name)

2023-01-29 19:33:37,055 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-29 19:33:37,057 INFO sqlalchemy.engine.Engine SELECT wafer.id AS wafer_id, wafer.created AS wafer_created, wafer.updated AS wafer_updated, wafer.serial_number AS wafer_serial_number, wafer.name AS wafer_name, wafer.description AS wafer_description 
FROM wafer
2023-01-29 19:33:37,057 INFO sqlalchemy.engine.Engine [generated in 0.00042s] ()
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
2023-01-29 19:33:37,060 INFO sqlalchemy.engine.Engine SELECT wafer.name AS wafer_name, wafer.serial_number AS wafer_serial_number 
FROM wafer
2023-01-29 19:33:37,060 INFO sqlalchemy.engine.Engine [generated in 0.00051s] ()
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
12 ABC
2023-01-29 19:33:37,064 INFO sqlalchemy.engine.Engine SELECT re

Manual SQL commands may naturally be used as well.

In [6]:
# Notice how this is different from session
with engine.connect() as connection:

    cursor = connection.execute(text("SELECT * FROM wafer WHERE name IS 12"))
    for row in cursor:
        print(row)

2023-01-29 19:33:37,144 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-29 19:33:37,145 INFO sqlalchemy.engine.Engine SELECT * FROM wafer WHERE name IS 12
2023-01-29 19:33:37,145 INFO sqlalchemy.engine.Engine [generated in 0.00099s] ()
(1, '2023-01-22 16:52:05', '2023-01-22 16:52:05', 'ABC', '12', None)
(2, '2023-01-22 16:52:25', '2023-01-22 16:52:25', 'ABC', '12', None)
(3, '2023-01-22 16:54:49', '2023-01-22 16:54:49', 'ABC', '12', None)
(4, '2023-01-29 13:11:56', '2023-01-29 13:11:56', 'ABC', '12', None)
(5, '2023-01-29 13:11:59', '2023-01-29 13:11:59', 'ABC', '12', None)
(6, '2023-01-29 13:12:00', '2023-01-29 13:12:00', 'ABC', '12', None)
(7, '2023-01-29 13:12:00', '2023-01-29 13:12:00', 'ABC', '12', None)
(8, '2023-01-29 13:12:01', '2023-01-29 13:12:01', 'ABC', '12', None)
(9, '2023-01-29 13:12:01', '2023-01-29 13:12:01', 'ABC', '12', None)
(10, '2023-01-29 13:12:01', '2023-01-29 13:12:01', 'ABC', '12', None)
(11, '2023-01-29 13:12:02', '2023-01-29 13:12:02', 'ABC', '12', No

### Adding simulation results

In this section TODO

Todo SQLModel class for holding S parameter, or n mode results?

In [7]:
import gdsfactory.simulation.gtidy3d as gt

with Session(engine) as session:

    for wavelength in (1.2, 1.4, 1.55):

        strip = gt.modes.Waveguide(
            wavelength=1.55,
            wg_width=0.5,
            wg_thickness=0.22,
            slab_thickness=0.0,
            ncore="si",
            nclad="sio2",
        )
        strip.compute_modes()
        strip.schema()

        # gm.ComputedResult(
        #     strip.neffs, strip.nmodes
        # )

        session.add(gm.Result(name='WG', type='Waveguide', value=strip))

    session.commit()

[19:33:38] INFO     Using client version: 1.8.1                               ]8;id=257025;file://c:\Users\Niko\AppData\Local\Programs\Python\Python310\lib\site-packages\tidy3d\__init__.py\__init__.py]8;;\:]8;id=866526;file://c:\Users\Niko\AppData\Local\Programs\Python\Python310\lib\site-packages\tidy3d\__init__.py#112\112]8;;\

TypeError: issubclass() arg 1 must be a class

Interesting queries might include filtering numerical quantities.

In [ ]:
with Session(engine) as session:

    # here .all() returns other data than the name as well
    for row in session.query(gm.ComputedResult.name.label("TODO")).all():
        print(row)

    for row in session.query(gm.ComputedResult.value).filter(
        gm.ComputedResult.value >= 2
    ).all():
        print(row)

2023-01-29 16:22:15,136 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-01-29 16:22:15,137 INFO sqlalchemy.engine.Engine SELECT computed_result.name AS "TODO" 
FROM computed_result
2023-01-29 16:22:15,137 INFO sqlalchemy.engine.Engine [cached since 1638s ago] ()
2023-01-29 16:22:15,138 INFO sqlalchemy.engine.Engine SELECT computed_result.value AS computed_result_value 
FROM computed_result 
WHERE computed_result.value >= ?
2023-01-29 16:22:15,139 INFO sqlalchemy.engine.Engine [cached since 722s ago] (2,)
2023-01-29 16:22:15,140 INFO sqlalchemy.engine.Engine ROLLBACK
